In [3]:
'''
Select an individual feature by attribute value from one feature class, then select features from another feature class by distance from the original feature. 
Originally used for selecting census block groups within a specified distance of a point feature. 
'''
def select_feature_by_attribute(focus_features, focus_field, focus_value, geo_features, selection_distance, output_path, overwrite): 
    
    # Construct SQL 
    sql = f"{focus_field} = '{focus_value}'"

    # Apply attribute selection 
    focus_selection, count          = arcpy.management.SelectLayerByAttribute(focus_features, 'NEW_SELECTION', sql)
    if count == '0': 
        print('No focus features selected')
    elif count != '1': 
        print(f'{count} focus features found')
    else: 
        print('Focus feature selected')
    
    # Select by distance 
    target_selection, merge, count  = arcpy.management.SelectLayerByLocation(geo_features, 'WITHIN_A_DISTANCE', focus_features, selection_distance, 'NEW_SELECTION')
    print(f'Selected  : {count} features')
    
    # Export target selection 
    if arcpy.Exists(output_path) and overwrite == True or arcpy.Exists(output_path) == False: 
        print(f'Exporting : {output_path}')
        arcpy.conversion.ExportFeatures(target_selection, output_path)
    elif arcpy.Exists(output_path) and overwrite == False: 
        print(f'Feature class exists: {output_path}')
    
    return output_path

In [4]:
'''
Parameters
'''

import os 

# Focus inputs 
focus_features     = r'C:\DALE\Personal\Minnesota\MyProject\ACS.gdb\Focus_Points'
focus_field        = 'NAME'
focus_value        = 'Strube'

# Target inputs 
geo_features       = r'C:\DALE\Personal\Minnesota\MyProject\SQLServer-IT-GISDB-monroegis(gisdata).sde\monroegis.DBO.TAXPARMaster_Master2024'
selection_distance = '6 Miles'

# Output 
output_folder      = r'C:\DALE\Personal\Minnesota\GIS Projects\ACS\Output Data\Block Group Shapefiles'
output_name        = focus_value + "_" + selection_distance.replace(" ", "")
geo_output         = os.path.join(output_folder, output_name + '.shp')
overwrite          = True 

'''
Run 
'''
geo_output = select_feature_by_attribute(focus_features, focus_field, focus_value, geo_features, selection_distance, geo_output, overwrite)

Focus feature selected
Selected  : 15703 features
Exporting : C:\DALE\Personal\Minnesota\GIS Projects\ACS\Output Data\Block Group Shapefiles\Strube_6Miles.shp
